In [ ]:
!pip install praw
!pip install asyncpraw
!pip install langdetect

In [ ]:
import pandas as pd
from datasets import load_dataset
import os

import praw
from time import sleep

from langdetect import detect, DetectorFactory

In [ ]:
# Parameters
TARGET_PER_LABEL = 2200
MAX_LENGTH = 800             # max characters per post
SLEEP_TIME = 0.2             # seconds between requests to avoid rate limit
DetectorFactory.seed = 0  # for consistent results

# Part 2   
fetching 4 class from reddit


In [ ]:
df_hf = pd.read_csv('/content/huggingFace_mental_health_dataset.csv')
save_file = "reddit_mental_health_dataset.csv"

if os.path.exists(save_file):
    df_existing = pd.read_csv(save_file)
    data_reddit = df_existing.to_dict("records") + df_hf.to_dict("records")
    seen_texts = set(df_existing["body"].tolist()) | set(df_hf["body"].tolist())  # avoid duplicates
    print(f"🔄 Resuming... already have {len(df_existing)} posts")
else:
    data_reddit = df_hf.to_dict("records")
    seen_texts = set(df_hf["body"].tolist())

In [ ]:
# 1. Setup Reddit API
reddit = praw.Reddit(client_id="YOUR_CLIENT_ID",
                     client_secret="YOUR_CLIENT_SECRET",
                     user_agent="YOUR_USER_AGENT")

In [ ]:
# 2. Define rest of subreddits

mental_subreddits = {
    "Anxiety": {
        "sub_list": ["anxiety", "socialanxiety", "PanicAttack", "HealthAnxiety"],
        "MIN_LENGTH": 80
    },
    "Suicidal": {
        "sub_list": ["SuicideWatch", "Suicidal_Thoughts", "SuicideBereavement"],
        "MIN_LENGTH": 50
    },
    "Addiction": {
        "sub_list": ["StopSmoking", "addiction", "StopDrinking", "stopgaming"],
        "MIN_LENGTH": 80
    },
    "Normal": {
        "sub_list": ["happy", "CasualConversation", "wholesomememes", "LifeProTips"],
        "MIN_LENGTH": 50
    }
}


In [ ]:
def is_english(text):
    try:
        return detect(text) == "en"
    except:
        return False

In [ ]:
def fetch_posts_for_label(label, subreddits, MIN_LENGTH):

    for subreddit_name in subreddits:
        try:
            subreddit = reddit.subreddit(subreddit_name)
            print(f"\nFetching '{label}' posts from r/{subreddit_name}...")

            for post in subreddit.new(limit=None):
                title = post.title.strip()
                body = post.selftext.strip()

                if len(body) < MIN_LENGTH or len(body) > MAX_LENGTH or body in seen_texts:
                    continue
                if not is_english(body):
                    continue

                seen_texts.add(body)

                data_reddit.append({
                    "post_id": post.id,
                    "subreddit": subreddit_name,
                    "title": title,
                    "body": body,
                    "label": label
                })

                sleep(SLEEP_TIME)

            # Save after each subreddit
            df_reddit = pd.DataFrame(data_reddit)
            df_reddit.to_csv(save_file, index=False)
            print(f"💾 Progress saved: {len(df_reddit)} posts so far")

            current_label_count = sum(1 for d in data_reddit if d["label"] == label)
            if current_label_count >= TARGET_PER_LABEL:
              return

        except Exception as e:
            print(f"⚠️ Could not fetch from r/{subreddit_name}: {e}")


In [ ]:
# Example fetch call
for label, subreddits in mental_subreddits.items():
    fetch_posts_for_label(label, subreddits["sub_list"], subreddits["MIN_LENGTH"])


In [ ]:
df_reddit = pd.DataFrame(data_reddit)
df_reddit = df_reddit.drop_duplicates(subset="body").reset_index(drop=True)
df_reddit['label'].value_counts()

In [ ]:
BALANCE_SIZE = 2000

balanced_df = (
    df_reddit
    .groupby("label", group_keys=False)   # group by label
    .apply(lambda x: x.sample(n=min(BALANCE_SIZE, len(x)), random_state=42))  # sample within each class
    .reset_index(drop=True)
)

balanced_df["label"].value_counts()

In [ ]:
from google.colab import files
balanced_df.to_csv(save_file, index=False)
files.download(save_file)